# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [16]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

Why do I get an error when I set up the code like this? 

In [17]:
prestige_admit = pd.crosstab(index=df['admit'], 
                            columns= df['prestige'],
                            rownames= ["not admitted","admitted"],
                            colnames= ["prestige1", "prestige2","prestige3","prestige4"] 
                            )

AssertionError: arrays and names must have the same length

In [18]:
# frequency table for prestige and whether or not someone was admitted

prestige_admit = pd.crosstab(index=df["admit"], 
                           columns=df["prestige"])

prestige_admit.index= ["not admitted","admitted"]
prestige_admit.columns = ["prestige1", "prestige2", "prestige3", "prestige4"]
prestige_admit

,prestige1,prestige2,prestige3,prestige4
not admitted,28,95,93,55
admitted,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [19]:
dummy_ranks=(pd.get_dummies(df['prestige'], prefix='prestige_'))
print (dummy_ranks.head())

   prestige__1.0  prestige__2.0  prestige__3.0  prestige__4.0
0              0              0              1              0
1              0              0              1              0
2              1              0              0              0
3              0              0              0              1
4              0              0              0              1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3
#dummy_ranks=(pd.get_dummies(df['prestige'], drop_first=True, prefix='prestige_'))

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [20]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige__1':])
print (handCalc.head())

   admit    gre   gpa  prestige__1.0  prestige__2.0  prestige__3.0  \
0      0  380.0  3.61              0              0              1   
1      1  660.0  3.67              0              0              1   
2      1  800.0  4.00              1              0              0   
3      1  640.0  3.19              0              0              0   
4      0  520.0  2.93              0              0              0   

   prestige__4.0  
0              0  
1              0  
2              0  
3              1  
4              1  


In [21]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

prestige1_admit = pd.crosstab(index= handCalc["prestige__1.0"] , 
                           columns=handCalc["admit"])


prestige1_admit.columns= ["not admitted","admitted"]
prestige1_admit.index= ["not highest prestige", "highest prestige"]

prestige1_admit

,not admitted,admitted
not highest prestige,243,93
highest prestige,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [22]:
odds_admitted_high_prestige = 33/28

print(odds_admitted_high_prestige)

1.1785714285714286


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [23]:
odds_admitted_not_high_prestige= 93/243

print (odds_admitted_not_high_prestige)

0.38271604938271603


#### 3.3 Calculate the odds ratio

In [24]:
odds_ratio= odds_admitted_high_prestige/odds_admitted_not_high_prestige

print(odds_ratio)

3.079493087557604


#### 3.4 Write this finding in a sentenance: 

Answer:
 The odds of admission into grad schools for those who did not attend a #1 ranked undergrad are 3 times less than for those who attended a #1 ranked undergrad school. 

#### 3.5 Print the cross tab for prestige_4

In [25]:
prestige4_admit = pd.crosstab(index=handCalc["prestige__4.0"], 
                           columns=handCalc["admit"])

prestige4_admit.columns= ["not admitted","admitted"]
prestige4_admit.index= ["not lowest prestige", "lowest prestige"]

prestige4_admit

,not admitted,admitted
not lowest prestige,216,114
lowest prestige,55,12


#### 3.6 Calculate the OR 

In [26]:
odds_admitted_low_prestige = 12/55
odds_admitted_not_low_prestige= 114/216
odds_ratio= odds_admitted_low_prestige/odds_admitted_not_low_prestige

print(odds_ratio)

0.4133971291866028


#### 3.7 Write this finding in a sentence

Answer: 
The odds of admission into grad school for those who attended a lowest rank school are 41% less than for those who did not attend a lowest ranked undergrad school.

## Part 4. Analysis

##### Set 1 (aka most prestigious) as your reference category and merge prestige_2, prestige_3 and prestige_4 back into the dataset.

Reminder- How to use dummy variables to represent an n-category variable:
1. First note that we use a set of n-1 dummy variables as tools to represent an n‑category variable.
2. Choose one of the categories to serve as the “reference” category, the category to which you compare the other categories.
3. Create dummy (0/1) variables to represent each of the other categories. Each dummy is coded so that it has the value 1 if a case is in that category, and 0 if not.
4. Interpret the regression coefficient for each dummy variable as how that category compares to the reference category. (source)

In [27]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige__2':])

print (data.head())

   admit    gre   gpa  prestige__2.0  prestige__3.0  prestige__4.0
0      0  380.0  3.61              0              1              0
1      1  660.0  3.67              0              1              0
2      1  800.0  4.00              0              0              0
3      1  640.0  3.19              0              0              1
4      0  520.0  2.93              0              0              1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [28]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [29]:
train_cols = ['gre', 'gpa', 'prestige__2.0', 'prestige__3.0','prestige__4.0','intercept']

#### 4.2 Fit the model

In [30]:
model = sm.Logit(data['admit'], data[train_cols])
result= model.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [31]:
print(result.summary2())

                        Results: Logit
Model:              Logit            No. Iterations:   6.0000  
Dependent Variable: admit            Pseudo R-squared: 0.082   
Date:               2018-04-24 15:20 AIC:              467.6399
No. Observations:   397              BIC:              491.5435
Df Model:           5                Log-Likelihood:   -227.82 
Df Residuals:       391              LL-Null:          -248.08 
Converged:          1.0000           Scale:            1.0000  
---------------------------------------------------------------
                 Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
---------------------------------------------------------------
gre              0.0022   0.0011  2.0280 0.0426  0.0001  0.0044
gpa              0.7793   0.3325  2.3438 0.0191  0.1276  1.4311
prestige__2.0   -0.6801   0.3169 -2.1459 0.0319 -1.3013 -0.0589
prestige__3.0   -1.3387   0.3449 -3.8819 0.0001 -2.0146 -0.6628
prestige__4.0   -1.5534   0.4175 -3.7211 0.0002 -2.3716 -0.7352
i

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [32]:
params = result.params 
np.exp(params)

gre              1.002221
gpa              2.180027
prestige__2.0    0.506548
prestige__3.0    0.262192
prestige__4.0    0.211525
intercept        0.020716
dtype: float64

In [33]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print(np.exp(conf))

                   2.5%     97.5%        OR
gre            1.000074  1.004372  1.002221
gpa            1.136120  4.183113  2.180027
prestige__2.0  0.272168  0.942767  0.506548
prestige__3.0  0.133377  0.515419  0.262192
prestige__4.0  0.093329  0.479411  0.211525
intercept      0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of being admitted to graduate schools decrease by about 50% for those who attended a second ranked undergrad school. 

#### 4.6 Interpret the OR of GPA

Answer: For a one unit increase in gpa, the odds of being admitted to graduate school (versus not being admitted) increase by 123%.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [34]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size,1):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [35]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[220.         284.44444444 348.88888889 413.33333333 477.77777778
 542.22222222 606.66666667 671.11111111 735.55555556 800.        ]
[2.26       2.45333333 2.64666667 2.84       3.03333333 3.22666667
 3.42       3.61333333 3.80666667 4.        ]


#### 5.1 Recreate the dummy variables

In [36]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks=(pd.get_dummies(combos['prestige'], prefix='prestige_'))
print (dummy_ranks.head())

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.loc[:, 'prestige__2':])

print (combos.head())

   prestige__1.0  prestige__2.0  prestige__3.0  prestige__4.0
0              1              0              0              0
1              0              1              0              0
2              0              0              1              0
3              0              0              0              1
4              1              0              0              0
     gre       gpa  prestige  intercept  prestige__2.0  prestige__3.0  \
0  220.0  2.260000       1.0        1.0              0              0   
1  220.0  2.260000       2.0        1.0              1              0   
2  220.0  2.260000       3.0        1.0              0              1   
3  220.0  2.260000       4.0        1.0              0              0   
4  220.0  2.453333       1.0        1.0              0              0   

   prestige__4.0  
0              0  
1              0  
2              0  
3              1  
4              0  


#### 5.2 Make predictions on the enumerated dataset

In [40]:
combos['admit_pred'] = result.predict(combos[train_cols])

print (combos.tail())
len(combos)

       gre       gpa  prestige  intercept  prestige__2.0  prestige__3.0  \
395  800.0  3.806667       4.0        1.0              0              0   
396  800.0  4.000000       1.0        1.0              0              0   
397  800.0  4.000000       2.0        1.0              1              0   
398  800.0  4.000000       3.0        1.0              0              1   
399  800.0  4.000000       4.0        1.0              0              0   

     prestige__4.0  admit_pred  
395              1    0.334286  
396              0    0.734040  
397              0    0.582995  
398              0    0.419833  
399              1    0.368608  


400

#### 5.3 Interpret findings for the last 4 observations

Answer: 
Among those who had attended a number one ranked school, the probability of admission is 73%.
Among those who had attended a number two ranked school, the probability of admission is 58%.
Among those who had attended a number three ranked school, the probability of admission is 42%.
Among those who had attended a number four ranked school, the probability of admission is 37%.


The last 4 observations show that with all other variables constant (gre and gpa), the probability of being admitted into grad school is higher for those who attended a top ranked undergrad schools than for those who attended lower ranked undergrad schools. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.